In [4]:
# Importar las librerías necesarias
import sys
sys.path.insert(1, '/home/eatitall_scripts')
sys.path.insert(1, '/home/root/pctobs/lib/python3.8/site-packages')
import pandas as pd
from extraccion_informacion.clustering import clustering
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [5]:
csv_path='./../../archivos/datos_con_10_ejemplos_reglas_y_extraccion_entidades.csv'
# Cargamos el archivo CSV
df = pd.read_csv(csv_path)

In [6]:
def listar_columnas_string(df):
    columnas_string = [col for col in df.columns if df[col].dtype == 'object']
    return columnas_string

def listar_columnas_si_no(df):
    columnas_si_no = []
    for col in df.columns:
        if df[col].dtype == 'object':  # Verificar si la columna es de tipo object (potencialmente string)
            valores_unicos = df[col].dropna().unique()  # Obtener valores únicos sin contar NaN
            if set(valores_unicos) <= {'Sí', 'No'}:  # Verificar si todos los valores son 'Sí' o 'No'
                columnas_si_no.append(col)
    return columnas_si_no

def convertir_si_no_a_dummy(df, columnas):
    for columna in columnas:
        df[columna] = df[columna].map({'Si': 1, 'No': 0})
    return df

def eliminar_elementos(lista_principal, elementos_a_eliminar):
    """
    Devuelve una nueva lista que contiene los elementos de 'lista_principal',
    excluyendo los elementos que se encuentran en 'elementos_a_eliminar'.

    Parámetros:
    lista_principal: Lista de elementos de la cual eliminar elementos.
    elementos_a_eliminar: Lista de elementos a eliminar de la lista principal.
    """
    return [elemento for elemento in lista_principal if elemento not in elementos_a_eliminar]

def convertir_si_no_a_dummy(df, columnas):
    for columna in columnas:
        df[columna] = df[columna].map({'Si': 1, 'No': 0})
    return df

In [7]:
lista_principal=listar_columnas_string(df)
elementos_a_eliminar=listar_columnas_si_no(df)
variables_string_no_bol=eliminar_elementos(lista_principal, elementos_a_eliminar)


columnas=listar_columnas_si_no(df)
df_dummies=convertir_si_no_a_dummy(df,columnas)
df_con_imc_con_reglas_con_dummy_sin_strings=df_dummies
df_con_imc_con_reglas_con_dummy_sin_strings.drop(variables_string_no_bol, axis=1, inplace=True)

In [9]:

# Preparación de datos
# Supongamos que 'data' es tu DataFrame y 'target' es la columna objetivo
X = df_con_imc_con_reglas_con_dummy_sin_strings.drop(['prediabetes','diabetes_con_cv','diabetes_con_hipertension',
                                    'diabetes_con_lipidos','diabetes_con_estatinas','diabetes_mayores_de_65',
                                    'diabetes_mayores_de_65_y_salud_saludable','diabetes_mayores_de_65_y_salud_compleja',
                                    'diabetes_mayores_de_65_y_salud_muy_compleja'], axis=1)  # Características
y = df_con_imc_con_reglas_con_dummy_sin_strings[['prediabetes','diabetes_con_cv','diabetes_con_hipertension',
                                    'diabetes_con_lipidos','diabetes_con_estatinas','diabetes_mayores_de_65',
                                    'diabetes_mayores_de_65_y_salud_saludable','diabetes_mayores_de_65_y_salud_compleja',
                                    'diabetes_mayores_de_65_y_salud_muy_compleja']]               # Variable objetivo

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo Random Forest
clf = DecisionTreeClassifier(max_depth=7, min_samples_split=4, min_samples_leaf=2, random_state=42)

# Entrenar el modelo
clf.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = clf.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').